# Analítica de datos
# Examen parcial 1
# 2019-02

# <font color='red'> Alejandro Narvaez, A00130548 </font>

# <font color='red'>ENTREGA PRIMERA PARTE</font> - <font color='blue'> EN CLASE </font>

Usted es el encargado de analítica de una empresa de telefonía celular y tiene que proporcionar soluciones para hacer frente a las problemáticas de un sector que ha llegado a saturación del mercado. Tanto su empresa como sus competidores directos tienen que disputarse por una base de clientes limitada, de tal forma que usted tiene que responder a un objetivo estratégico definido por la dirección así:  

    "Mantener y fidelizar a nuestros clientes por medio de un servicio de calidad que se adapte a sus necesidades particulares."
    
Su compañía dispone de una base de datos histórica de personas que hace un año eran clientes propios. Algunos de esos clientes siguen siéndolo hoy en día, otros ya no lo son. La idea es podeer identificar los clientes que son propensos a dejar la compañía, para poder pensar en programas de fidelización preventivos.

Los campos del dataset son los siguientes:
1.  ID: Código identificador de los clientes de la compañía de telefonía móvil
1.	ESTADO: Describe si el usuario sigue con la compañía (VINCULADO) o no (RETIRADO)
1.	INGRESOS: Promedio de ingresos del cliente en pesos
1.	CASA: Precio de la casa en la que vive el cliente en pesos
1.	PRECIO_DISPOSITIVO: Precio del celular del cliente en pesos
1.  GÉNERO: "Hombre" o "Mujer"
1.	MESES: Antigüedad del usuario en meses
1.	DURACION: Promedio de duración de las llamadas hechas por el cliente en minutos
1.	SOBRECARGO: Promedio de minutos que se sobrepasa el usuario en un mes
1.	SALDO_RESTANTE: Promedio de minutos de su plan que le quedan al usuario sin utilizar cada mes
1.	SATISFACCION: nivel de satisfacción del usuario de 0 a 10 (muy satisfecho), obtenido a partir de una encuesta.

La idea es poder predecir el ESTADO a partir de las otras variables, utilizando modelos de aprendizaje supervisado (KNN, NaiveBayes, y regresión logística).

# 1. Entendimiento de los datos, limpieza

El archivo "DatosTelco.csv" contiene el dataset que tienen que analizar.
Se recomienda abrirlo primero en un lector de archivos planos para entender preliminarmente su formato y así poderlo cargar adecuadamente con Python.

Teniendo en cuenta el tipo de problema en cuestión (clasificación o regresión), realice un análisis exploratorio de los datos estableciendo el baseline, verificando la calidad de los datos (tipos de las variables, valores inválidos, excepciones, valores faltantes, etc.), utilizando gráficos para poder entender las distribuciones de los datos e identificar posibles problemas.

Tenga en cuenta lo siguiente: 
- para modificar ciertos valores de un dataframe, se utiliza "df.replace('oldvalue', 'newvalue')", si se trata de un valor NaN, se utiliza *np.nan* (ya sea el oldvalue o el newvalue)
- para cambiar un tipo de dato a numérico en una estructura de pandas, se utiliza su método ".astype('float64')"
- para negar una condición en python se utiliza el símbolo "~"
- para obtener una tabla de frecuencias de los registros (filas) con respecto al valor de una variable categórica se utiliza "pd.crosstab(index=df['var'], columns="conteo")
- para cambiar los valores de una columna en un dataframe con condiciones, utilizar "df.loc[condicion, 'columna']=newVal"
- cuando haya atributos con demasiados valores faltantes, pueden eliminar la columna correspondiente.
- cuando haya atributos con unos pocos valores faltantes, pueden eliminar los registros correspondientes.
- cuando el número de valores faltantes de un atributo no sea tan elevado, pero si sea considerable, pueden reemplazar los valores faltantes:
  - Si se trata de una variable categórica, pueden crear un nuevo valor, o reemplazar por la categoría más común
  - Si se trata de una variable numérica, pueden reemplazar por el promedio de los valores presentes del atributo
- cuando se quiere ordenar un dataframe por los valores de una columna se usa "df.sort_values("COLUMNA", ascending=True)"
- para borrar los registros a partir de un índice de fila se utiliza "df.drop([0,3])"
- para borrar los registros a partir de una condición se utiliza "df = df[df.edad<99]"
- para borrar una columna por nombre se utiliza --> df = df.drop('columna',axis=1) 

# Puntos a desarrollar

DURANTE EL EXAMEN, EN CLASE: 
1. <font color='red'>Carguen el archivo en memoria y exploren los datos. Antes de hacer limpieza identifiquen, el baseline global (0.1) y los baselines por GÉNERO (0.2).
   ¿A primera vista, solo considerando el género, cree que es una buena idea crear un modelo predictivo de la deserción de hombres y de mujeres de manera separada? (0.2).</font>
2. <font color='red'>Identifiquen los problemas e inconsistencias que tienen los datos, teniendo en cuenta el diccionario de datos y el contexto del problema. Limpien los datos, argumentando las razones de cada transformación o eliminación de datos.
(1.1). </font>

EN CASA, PARA ENTREGAR: 
3. <font color='red'>Para arreglar un problema existente en la variable CASA: (1.0)
    * utilice un modelo de regresión lineal (use el modelo sklearn.linear_model.LinearRegression, no use statsmodels)
    * considere solo las demas variables NUMÉRICAS como variables predictivas
    * cree el mejor modelo que utilice 1 solo variable, calcule el R2 ajustado e interprételo
    * utilice holdout (70/30) como protocolo de evaluación
</font>

# 1.1 Carga y baselines pre-limpieza de datos

In [84]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression # modelos lineales
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import cross_val_score #protocolo de evaluación
from sklearn import datasets, metrics
from sklearn import preprocessing
import seaborn as sns

In [85]:
df = pd.read_csv('DatosTelco.csv', sep=';')

In [86]:
df

,ID,ESTADO,INGRESOS,CASA,PRECIO_DISPOSITIVO,GENERO,MESES,DURACION,SOBRECARGO,SALDO_RESTANTE,SATISFACCION
0,1,VINCULADO,4074840,6.878215e+08,1444153.0,Mujer,26,2.3,11.167542,20.3,1.762872
1,2,VINCULADO,8574088,1.349120e+09,2157661.0,Hombre,23,3.7,0.000000,53.5,3.361266
2,3,RETIRADO,795993,1.326558e+08,452809.0,Mujer,38,16.8,350.610224,13.9,8.359438
3,4,RETIRADO,8115015,1.290405e+09,2617184.0,Mujer,37,17.6,230.913888,45.7,8.964017
4,5,RETIRADO,8187499,1.305348e+09,666069.0,Mujer,19,0.0,131.541983,67.2,8.808206
5,6,VINCULADO,7250225,1.183512e+09,548116.0,Hombre,19,2.2,0.000000,72.0,1.979549
6,7,RETIRADO,7256758,1.193231e+09,3493707.0,Mujer,20,14.8,217.322554,69.5,8.285137
7,8,RETIRADO,3262245,5.088212e+08,985708.0,Mujer,32,1.3,275.595131,19.0,8.040629
8,9,VINCULADO,7281977,1.185930e+09,2453442.0,Mujer,41,3.0,0.000000,64.6,2.654332
9,10,RETIRADO,874177,1.675603e+08,1403586.0,Mujer,19,17.8,0.000000,19.9,2.138861


In [87]:
df.describe()

,ID,INGRESOS,CASA,PRECIO_DISPOSITIVO,MESES,DURACION,SOBRECARGO,SALDO_RESTANTE,SATISFACCION
count,23162.000000,2.316200e+04,2.306200e+04,2.316200e+04,23162.000000,23162.000000,23159.000000,23162.000000,23162.000000
mean,11581.500000,4.813115e+06,7.823571e+08,1.454784e+06,23.862620,9.609852,110.050331,42.991888,5.480840
std,6686.437803,2.737374e+06,5.540029e+08,1.427590e+07,14.816863,7.040007,100.769297,30.204540,2.809938
min,1.000000,1.500000e+05,-1.779042e+07,1.693380e+02,-55.000000,0.000000,0.000000,0.000000,0.613580
25%,5791.250000,2.156296e+06,3.660683e+08,6.571840e+05,14.000000,2.100000,12.568985,15.200000,2.705504
50%,11581.500000,6.137553e+06,9.703263e+08,1.237542e+06,24.000000,14.600000,75.379060,37.650000,4.064550
75%,17371.750000,7.347180e+06,1.171952e+09,1.829751e+06,34.000000,16.100000,201.529547,70.000000,8.268244
max,23162.000000,9.650000e+06,4.664422e+10,1.859365e+09,79.000000,20.100000,411.168400,130.700000,9.642618


In [88]:
df.mean()

ID                    1.158150e+04
INGRESOS              4.813115e+06
CASA                  7.823571e+08
PRECIO_DISPOSITIVO    1.454784e+06
MESES                 2.386262e+01
DURACION              9.609852e+00
SOBRECARGO            1.100503e+02
SALDO_RESTANTE        4.299189e+01
SATISFACCION          5.480840e+00
dtype: float64

### Baseline global

In [89]:
total_filas= df.shape[0]
total_filas_vinculado=df[df['ESTADO'] == "VINCULADO"].shape[0]
result= total_filas_vinculado/total_filas
result=result*100
print("El baseline de los CLIENTES vinculados es de "+str(result)+"%")
print("El proximo cliente que vaya a entrar probablemente su estado pase a RETIRADO")

El baseline de los CLIENTES vinculados es de 49.63733701752871%
El proximo cliente que vaya a entrar probablemente su estado pase a RETIRADO


### Baselines por género

In [90]:
#Mujeres vinculadas
V=df[df['ESTADO'] == "VINCULADO"]
total_V=V.shape[0]
total_filas_vinculado_y_mujer=V[V['GENERO']=="Mujer"].shape[0]
total_MV=(total_filas_vinculado_y_mujer/total_V)*100

#Hombres vinculados

total_filas_vinculado_y_hombre=V[V['GENERO']=="Hombre"].shape[0]
total_HV=(total_filas_vinculado_y_hombre/total_V)*100
print("Hombres vinculados "+str(total_HV)+"%")
print("Mujeres vinculadas "+str(total_MV)+"%")


Hombres vinculados 50.36966165086544%
Mujeres vinculadas 49.630338349134554%


### Conclusión

No me parece buena idea hacer un modelo predictivo de la disercion de hombres y mujeres de forma separada. \
Mucho menos teniendo en cuenta que aun no se han limpiado los datos. \
Los hombres son mas fieles que las mujeres en el servicio de telefonia movil.
Por lo general hay mas hombres vinculados que mujeres vinculadas

# 1.2 Análisis de los problemas de calidad de datos

In [91]:
df=df.dropna() 
df

,ID,ESTADO,INGRESOS,CASA,PRECIO_DISPOSITIVO,GENERO,MESES,DURACION,SOBRECARGO,SALDO_RESTANTE,SATISFACCION
0,1,VINCULADO,4074840,6.878215e+08,1444153.0,Mujer,26,2.3,11.167542,20.3,1.762872
1,2,VINCULADO,8574088,1.349120e+09,2157661.0,Hombre,23,3.7,0.000000,53.5,3.361266
2,3,RETIRADO,795993,1.326558e+08,452809.0,Mujer,38,16.8,350.610224,13.9,8.359438
3,4,RETIRADO,8115015,1.290405e+09,2617184.0,Mujer,37,17.6,230.913888,45.7,8.964017
4,5,RETIRADO,8187499,1.305348e+09,666069.0,Mujer,19,0.0,131.541983,67.2,8.808206
5,6,VINCULADO,7250225,1.183512e+09,548116.0,Hombre,19,2.2,0.000000,72.0,1.979549
6,7,RETIRADO,7256758,1.193231e+09,3493707.0,Mujer,20,14.8,217.322554,69.5,8.285137
7,8,RETIRADO,3262245,5.088212e+08,985708.0,Mujer,32,1.3,275.595131,19.0,8.040629
8,9,VINCULADO,7281977,1.185930e+09,2453442.0,Mujer,41,3.0,0.000000,64.6,2.654332
9,10,RETIRADO,874177,1.675603e+08,1403586.0,Mujer,19,17.8,0.000000,19.9,2.138861


In [92]:
def drop_numerical_outliers(df, minq=0.01,maxq=0.99):
    # Constrains will contain `True` or `False` depending on if it is a value below the threshold.
    constrains = df.select_dtypes(include=[np.number]) \
        .apply(lambda x: x.between(x.quantile(minq), x.quantile(maxq)), reduce=False) \
        .all(axis=1)
    # Drop (inplace) values set to be rejected
    df.drop(df.index[~constrains], inplace=True)
drop_numerical_outliers(df)
df


C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions
  after removing the cwd from sys.path.
C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,ID,ESTADO,INGRESOS,CASA,PRECIO_DISPOSITIVO,GENERO,MESES,DURACION,SOBRECARGO,SALDO_RESTANTE,SATISFACCION
231,232,VINCULADO,2700037,4.017522e+08,1021660.0,Hombre,27,1.3,39.995340,25.7,2.668542
232,233,VINCULADO,7700681,1.258639e+09,1179766.0,Hombre,34,2.6,19.412611,79.8,2.720565
233,234,VINCULADO,3037281,4.705228e+08,1099960.0,Hombre,16,2.6,0.000000,19.8,2.772292
234,235,VINCULADO,1405579,2.767829e+08,1314343.0,Hombre,20,16.9,0.000000,29.7,2.697260
235,236,VINCULADO,3283476,5.626943e+08,818781.0,Mujer,0,1.2,0.000000,20.7,2.063530
236,237,RETIRADO,7412460,1.119748e+09,385461.0,Hombre,5,1.5,174.226214,61.6,7.906551
238,239,VINCULADO,6859606,1.090320e+09,355886.0,Hombre,23,16.4,0.000000,69.3,2.324773
239,240,RETIRADO,2843665,4.896594e+08,2136793.0,Hombre,34,1.0,34.071173,4.0,3.112052
240,241,RETIRADO,1899885,2.755053e+08,604226.0,Hombre,5,15.8,179.994408,19.9,7.842791
241,242,RETIRADO,7194206,1.162108e+09,2158469.0,Hombre,37,16.8,267.487135,69.1,8.049385


### Resumen de problemas encontrados, acciones tomadas:

1. [CASA] [valores negativos]: Elimine las variables negativas y los datos atipicos del data frame
2. [MESES] [valores negativos]: Elimine las variables negativas y los datos atipicos del data frame \
no valia la pena sacar la media y remplazar \
Elimine valores faltantes
                    
                    

# <font color='red'>ENTREGA SEGUNDA PARTE</font> - <font color='blue'> POST-CLASE </font>

# 1.3 Valores faltantes de CASA: modelo de regresión lineal usando holdout y forward stepwise

In [108]:
lista= ["INGRESOS","MESES",	"PRECIO_DISPOSITIVO","DURACION","SOBRECARGO","SALDO_RESTANTE","SATISFACCION"]
for i in lista:
    x=df[[i]]
    y=df[["CASA"]]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=12345)
    regressor=LinearRegression()
    regressor.fit(X_train,y_train)
    pred=regressor.predict(X_test)
    print(i)
    print("Coeficiente:",regressor.coef_[0][0])
    print("Intercepto:",regressor.intercept_[0])
    print("R2:",regressor.score(X_test,y_test))
    print("----------------------------------")
    

INGRESOS
Coeficiente: 154.59750402282353
Intercepto: 34908106.77296424
R2: 0.9933104676020573
----------------------------------
MESES
Coeficiente: -203038.32665400824
Intercepto: 778055064.6940163
R2: -0.0001966108689270829
----------------------------------
PRECIO_DISPOSITIVO
Coeficiente: 13.145034582389309
Intercepto: 756092881.9316186
R2: 0.0004344649627547881
----------------------------------
DURACION
Coeficiente: -9395122.0492598
Intercepto: 861774898.0215337
R2: 0.01763516244096286
----------------------------------
SOBRECARGO
Coeficiente: -11365.17600326824
Intercepto: 774450693.2479974
R2: -0.0003576190916851907
----------------------------------
SALDO_RESTANTE
Coeficiente: 11990150.92513126
Intercepto: 268004176.047324
R2: 0.7145693424728126
----------------------------------
SATISFACCION
Coeficiente: 115339.50988325507
Intercepto: 772593518.6371937
R2: -0.0002746292385440441
----------------------------------


# 2. Modelos de clasificación

## Puntos a desarrollar:

<font color='red'>El objetivo final es identificar los clientes más propensos a irse de la compañía, con el fin de poder realizar campañas de fidelización. Para tal propósito, se ha decidido buscar el mejor modelo entre **K-NN** y **Naïve Bayes**, pero **solo considerando las variables predictivas numéricas**. </font>

1. <font color='red'> Establezca el **protocolo de evaluación** y la métrica de evaluación más adecuados para la construcción de los modelos de clasificación (0.3)</font>

2. <font color='red'> Construya del mejor modelo **K-NN**, buscando el mejor valor de K (subir hasta un valor de K=25) (1.1)</font>

3. <font color='red'> Construya del mejor modelo **Naïve Bayes** (use la clase GaussianNB), buscando el mejor valor del suavizador de Laplace (var_smoothing) (0.7)</font>

4. <font color='red'> Compárelos (métricas, matriz de confusión), escoja el mejor, y concluya (0.3)</font>

# 2.1 Protocolo de evaluación

In [94]:
#from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, LeaveOneOut #Iteradores de C-V

#knn = neighbors.KNeighborsClassifier(n_neighbors=3)
#kf = KFold(n_splits=10, shuffle=True)
#acc_test_vec=[]
#for indices_train, indices_test in kf.split(x):
    #print("%s %s" % (indices_train, indices_test))
 #   print (indices_train)
  #  knn.fit(x[indices_train], y[indices_train])
   # y_pred = knn.predict(x[indices_test])
    #acc_test_vec.append(metrics.accuracy_score(y[indices_test], y_pred))  
#acc_test_vec

# 2.2 Modelo K-NN

In [122]:
ks=[1,3,5,7,9,11,15,17,22,25]
x = df[["INGRESOS","MESES",	"PRECIO_DISPOSITIVO","DURACION","SOBRECARGO","SALDO_RESTANTE","SATISFACCION","CASA"]].values
y = df[["ESTADO"]].values
#y = np.expand_dims(y, axis=1)
# X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=12345)
for k in ks:
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print(y_pred)
    print("Con K = ", k, ", exactitud: ", metrics.accuracy_score(y_test, y_pred))

C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


['RETIRADO' 'RETIRADO' 'RETIRADO' ... 'RETIRADO' 'RETIRADO' 'RETIRADO']
Con K =  1 , exactitud:  0.4976943346508564


C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


['VINCULADO' 'RETIRADO' 'VINCULADO' ... 'VINCULADO' 'RETIRADO' 'RETIRADO']
Con K =  3 , exactitud:  0.49538866930171277


C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


...

['RETIRADO' 'VINCULADO' 'VINCULADO' ... 'VINCULADO' 'RETIRADO' 'VINCULADO']
Con K =  17 , exactitud:  0.49061264822134387


C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


['RETIRADO' 'VINCULADO' 'VINCULADO' ... 'VINCULADO' 'RETIRADO' 'VINCULADO']
Con K =  22 , exactitud:  0.4939064558629776


C:\Users\Alejandro Narvaez\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


['RETIRADO' 'VINCULADO' 'VINCULADO' ... 'VINCULADO' 'RETIRADO' 'VINCULADO']
Con K =  25 , exactitud:  0.49209486166007904


#### 2.3 Modelo Naïve Bayes

In [ ]:
ks=[1,3,5,7,9,11,15,17,22,25]

#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=12345)
acc_test_vec=[]     
print("No Estudiantes")
for k in ks:
    knn2 = neighbors.KNeighborsClassifier(n_neighbors=k)
    for indices_train, indices_test in kf2.split(X_train2,y_train2):
        y_train2=y_train2.astype('int')
        y_train2=np.ravel(y_train2)
        knn2.fit(X_train2[indices_train], y_train2[indices_train])
        y_pred2 = knn2.predict(X_train2[indices_test])
        acc_test_vec.append([metrics.cohen_kappa_score(y_train2[indices_test].astype('int'), y_pred2),k])
        print("Con K = ", k, ", kappa: ", metrics.cohen_kappa_score(y_train2[indices_test].astype('int'), y_pred2))
    print("------------------------------------------")
max(acc_test_vec)

# 2.4 Comparación